## Connect to Google Drive and install packages 

In [ ]:
#run once per session
from google.colab import drive
drive.mount('/content/drive')

!pip install backports.cached-property
!pip install imagecodecs

## Fill out variables and load packages

In [ ]:
import glob
#Paths of tif files
TIF_PATHS = glob.glob('/content/drive/MyDrive/Guillaume_Shared/NucID/10X_trainingData/Images_masks/dense/img4.tif')

# enter values of um per pixel if using syd shaffer scope with 2x2 binning: 4X=3.25, 10X=1.29, 20X=.65, 60X=.22
RunPixelSize = 1.29

# enter the channel that contains nuclei, if there is only 1 channel enter 1
NucleusChannel = 1

# his is the directory to the weights you want to use
WeightsPath = '/content/drive/MyDrive/Colab/NucID/weights/10X_ms.pt'


#This is what level of confidence you want your model to have a nucleus is a nucleus to count it (ex. .2 means the model thinks there is a 20% chance there is a nucleus). you can also set the threshold to 0 and filter later
Confidence_thresh = .4

#This is the path to where all the packages are stored such as deeptile and yolov7
PackagePath = '/content/drive/MyDrive/Colab/NucID/packages'





### YOU SHOULD NOT NEED TO CHANGE THE PARAMETERS BELOW
# This is the tile size the model takes in. The tile size is ideally the same size of tiles the model was trained on. For the models provided the tile size should stay 640
tileSize=640

#Enter the um per pixel value of the images used for training
TrainPixelSize = 1.29

#This is how much overlap there are between tiles, this should stay at .1 for most applications
overlap=.1

## DO NOT CHANGE THES LINES OF CODE
#add path to packages
import sys
sys.path.append(PackagePath)
sys.path.append(str(PackagePath + '/yolov7'))

from NucID.Running_functions import Nucid, checkNucXY, FilterCoords

print(TIF_PATHS)
print("number of images to process: " + str(len(TIF_PATHS)) )

## Initate NucID object

In [ ]:
#initiate NucID object
nid = Nucid(nuc_channel=NucleusChannel, weights_path=WeightsPath, package_path=PackagePath, conf_thresh=Confidence_thresh, tileSize=tileSize, overlap=overlap, run_um_per_pix=RunPixelSize, train_um_per_pix=TrainPixelSize)

## Run NucID

In [ ]:
#Run the Model
for tif_path in TIF_PATHS:
  nid.RunNucID(tif_path)

Check if output coodinates are correct

In [ ]:
#Path to tif file you want to check
tifPath = nid.tif_path

#Path to coodinate path for the tif file specified above
coordPath = nid.coord_path

#Check the XY coordinates
checkNucXY(tif_path = tifPath, nucxy_path = coordPath, conf_thresh=0)

Filter output coodinates by confidence

In [5]:
#Path to coodinate you want to filter
coordPath = nid.coord_path
#Confidence threshold you want to filter by
Confidence_thresh = .4

#Filter the coodinates
FilterCoords(nucxy_path=coordPath, conf_thresh=Confidence_thresh)

## Test model

In [ ]:
#coodinate of center of tile you would like to test the model on
Coordinate = [500,630]

#Run fuction to test the model
%matplotlib inline
nid.TestModel(location = Coordinate ,min_brightness=.15,figSize=20)